<a href="https://colab.research.google.com/github/DiproHaque/Anomaly_Detection_Synthetic_Dataset/blob/main/Synthesizer_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:

qt = '/content/drive/MyDrive/Syn_DS/Profiles/'

amount_df = pd.read_excel(qt + 'Amount_Profile.xlsx')
device_df = pd.read_excel(qt + 'Device_Network_Profile.xlsx')
location_df = pd.read_excel(qt + 'Location_Profile.xlsx')
temporal_df = pd.read_excel(qt + 'Temporal_Profile.xlsx')
txn_type_df = pd.read_excel(qt + 'Txn_Type_Profile.xlsx')
text_df = pd.read_excel(qt + 'Texual_Profile.xlsx')
velocity_df = pd.read_excel(qt + 'Velocity_Profile.xlsx')

In [ ]:
#weighter random picker
def weighted_choice(df, value_col, weight_col):
    return random.choices(
        df[value_col].tolist(),
        weights=df[weight_col].tolist(),
        k=1
    )[0]

In [ ]:
#random date generator
def random_datetime():
    start = datetime(2024, 1, 1)
    end = datetime(2025, 12, 31)
    return start + (end - start) * random.random()


In [ ]:
#random trx_ID generator
def generate_txn_id():
    return "TXN" + str(random.randint(10**9, 10**10-1))


In [ ]:
#random account generator
def generate_account():
    return "AC" + str(random.randint(1000000000, 9999999999))


In [ ]:
#user base
TOTAL_USERS = 5000

users = []
for i in range(TOTAL_USERS):
    users.append({
        "User_ID": f"U{i+10000}",
        "Velocity_Type": weighted_choice(
            velocity_df, "User_Class", "User_%"
        )
    })

users_df = pd.DataFrame(users)


In [ ]:
transactions = []

for _, user in users_df.iterrows():

    txn_count = random.randint(5, 25)

    for _ in range(txn_count):

        txn_type = weighted_choice(txn_type_df, "Txn_Type", "Txn_%")
        country = weighted_choice(location_df, "Country", "Txn_%")

        device = weighted_choice(device_df, "Device_Type", "Txn_%") # Corrected column names

        txn_time = random_datetime()

        amount = round(random.uniform(50, 50000), 2)

        resp_code = random.choice(["00", "05", "12", "91"])

        transactions.append({
            "Txn_ID": generate_txn_id(),
            "User_ID": user["User_ID"],
            "Tran_Source": device,
            "Create_Date": txn_time,
            "Txn_Amount": amount,
            "From_Account": generate_account(),
            "To_Account": generate_account(),
            "Resp_Code": resp_code,
            "Tracer_No": random.randint(100000, 999999),
            "Txn_Type": txn_type,
            "Particulars": f"{txn_type} Transaction",
            "Remarks": "Successful" if resp_code == "00" else "Failed",
            "Device": device,
            "IP_Address": f"192.168.{random.randint(1,255)}.{random.randint(1,255)}",
            "Location": country,
            "Country": country,
            "city_by_coordinates": country,
            "city_name": country
        })


In [ ]:
for txn in transactions:
    risk = 0

    if txn["Txn_Amount"] > 30000:
        risk += 1
    if txn["Resp_Code"] != "00":
        risk += 1
    if txn["Device"] == "Unknown":
        risk += 1

    txn["Fraud_Label"] = 1 if risk >= 2 else 0


In [ ]:
final_df = pd.DataFrame(transactions)
final_df.to_csv("synthetic_transactions.csv", index=False)


In [6]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/Syn_DS/Profiles/'

amount_df = pd.read_excel(base_path + 'Amount_Profile.xlsx')
device_df = pd.read_excel(base_path + 'Device_Network_Profile.xlsx')
location_df = pd.read_excel(base_path + 'Location_Profile.xlsx')
temporal_df = pd.read_excel(base_path + 'Temporal_Profile.xlsx')
txn_type_df = pd.read_excel(base_path + 'Txn_Type_Profile.xlsx')
text_df = pd.read_excel(base_path + 'Texual_Profile.xlsx') # Corrected typo from text_profile.xlsx
velocity_df = pd.read_excel(base_path + 'Velocity_Profile.xlsx')

# Weighted random pick
def weighted_choice(df, value_col, weight_col):
    return random.choices(df[value_col].tolist(), weights=df[weight_col].tolist(), k=1)[0]

# Random datetime
def random_datetime():
    start = datetime(2024, 1, 1)
    end = datetime(2024, 12, 31)
    return start + (end - start) * random.random()

# Txn_ID generator
def generate_txn_id():
    return "TXN" + str(random.randint(10**9, 10**10-1))

# Account generator
def generate_account():
    return "AC" + str(random.randint(1000000000, 9999999999))

# Coordinate jitter ±0.003 ~ 300m
def jitter_coords(lat, lon):
    lat_jit = lat + np.random.uniform(-0.003, 0.003)
    lon_jit = lon + np.random.uniform(-0.003, 0.003)
    return lat_jit, lon_jit

TOTAL_USERS = 5000
users = []

for i in range(TOTAL_USERS):
    users.append({
        "User_ID": f"U{i+10000}",
        "Velocity_Type": weighted_choice(velocity_df, "Velocity_Type", "User_%")
    })

users_df = pd.DataFrame(users)

transactions = []

for _, user in users_df.iterrows():
    txn_count = random.randint(5, 25)

    for _ in range(txn_count):
        txn_type = weighted_choice(txn_type_df, "Txn_Type", "Txn_%")
        device = weighted_choice(device_df, "Device", "Usage_%")
        txn_time = random_datetime()
        amount = round(random.uniform(50, 50000),2)
        resp_code = random.choice(["00","05","12","91"])

        # Weighted city pick
        city = weighted_choice(location_df, "City", "Txn_%")

        # Base coordinates + jitter
        base_lat = city_coordinates.loc[city_coordinates['City']==city, 'Latitude'].values[0]
        base_lon = city_coordinates.loc[city_coordinates['City']==city, 'Longitude'].values[0]
        lat_jit, lon_jit = jitter_coords(base_lat, base_lon)

        # Fraud label logic
        risk = 0
        if amount > 30000:
            risk += 1
        if resp_code != "00":
            risk += 1
        if device == "Unknown":
            risk += 1
        fraud_label = 1 if risk >= 2 else 0

        transactions.append({
            "Txn_ID": generate_txn_id(),
            "User_ID": user["User_ID"],
            "Tran_Source": device,
            "Create_Date": txn_time,
            "Txn_Amount": amount,
            "From_Account": generate_account(),
            "To_Account": generate_account(),
            "Resp_Code": resp_code,
            "Tracer_No": random.randint(100000,999999),
            "Txn_Type": txn_type,
            "Particulars": f"{txn_type} Transaction",
            "Remarks": "Successful" if resp_code=="00" else "Failed",
            "Device": device,
            "IP_Address": f"192.168.{random.randint(1,255)}.{random.randint(1,255)}",
            "Location": city,
            "Country": "Bangladesh",
            "city_by_coordinates": f"{lat_jit},{lon_jit}",
            "city_name": city,
            "Fraud_Label": fraud_label
        })

synthetic_df = pd.DataFrame(transactions)

# Save to Drive
synthetic_df.to_csv('/content/drive/MyDrive/profiles/synthetic_transactions_fixed.csv', index=False)
print("Synthetic dataset with city & coordinates fix ready in Drive!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyError: 'Velocity_Type'